In [2]:
from six.moves import cPickle
import cv2
import numpy as np
import gym
from gym import wrappers

In [4]:
import sys, os
from functools import partial
from collections import defaultdict
sys.path.append('/home/etoestja/peas')
from peas.networks.rnn import NeuralNetwork
from peas.methods.neat import NEATPopulation, NEATGenotype

In [ ]:
class GYMGame(object):
    def __init__(self, game = 'Skiing-v0'):
        self.env = gym.make(game)
        self.actions = self.env.action_space.n
    # Resizing to black-white 42x42
    def resize_frame(self, frame):
        frame = frame[34:34+160, :160]
        # Resize by half, then down to 42x42 (essentially mipmapping). If
        frame = cv2.resize(frame, (10, 10))
        frame = frame.mean(2)
        frame = frame.astype(np.float32)
        frame *= (1.0 / 255.0)
        return frame
    # a = argmax_a Q(s,a)
    def predict_action(self, observation, network):
        #return env.action_space.sample()
        compressed_observation = self.resize_frame(observation).flatten()
        s = network.feed(compressed_observation)[-self.actions:]
        action = np.argmax(s)
        return(action)
    def solve(self, network):
        return False
    def evaluate(self, network):
        if not isinstance(network, NeuralNetwork):
            network = NeuralNetwork(network)
        
        observation = self.env.reset()
        done = False
        iteration, total_reward = 0, 0
        
        while not done:
            #env.render()
            action = self.predict_action(observation, network)
            observation, reward, done, info = self.env.step(action)
            total_reward += reward

            #if iteration % 500 == 0:
                #print(str(iteration))

            if total_reward < -12000 or iteration >= 8000:
                break

            iteration += 1

        
        res = {'fitness': total_reward, 'steps': iteration}
        return res
    def genotype(self):
        return lambda: NEATGenotype(inputs=100,
                                outputs = self.actions,
                                weight_range=(-3,3),
                                types=['tanh'])

In [ ]:
# Create a factory for genotypes (i.e. a function that returns a new
# instance each time it is called)
game = GYMGame()
genotype = game.genotype()

In [ ]:
# Create a population
pop = NEATPopulation(genotype, popsize=5, max_cores=8)

# Run the evolution, tell it to use the task as an evaluator
pop.epoch(generations=100, evaluator=game, solution=game)